In [6]:
import cv2
import mediapipe as mp
import numpy as np
import os

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [8]:
# 비디오 파일 이름
input_file = "D:\\Pitch\\sample_videos\\wooyoung_side.mp4" # 여기에 비디오 경로 넣으면 됩니다!

# 비디오 파일의 경우 이것을 사용하세요:
cap = cv2.VideoCapture(input_file)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [9]:
# 출력 파일 경로 및 이름 설정
output_file = os.path.splitext(input_file)[0] + "_dotted.mp4"
out = cv2.VideoWriter(output_file,cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

In [10]:
# 팔꿈치 관절 좌표화 코드
# left_elbow_coordinates = [] # 좌완투수
right_elbow_coordinates = [] # 우완투수, 위 리스트와 둘 중 하나는 주석처리해야 합니다.

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("비디오 파일을 처리하는데 실패했습니다.")
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        
        image_height, image_width, _ = image.shape

        if results.pose_landmarks:
            # left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            # cv2.circle(image, (int(left_elbow.x * image_width), int(left_elbow.y * image_height)), 5, (0, 255, 0), -1)
            # left_elbow_coordinates.append((left_elbow.x, left_elbow.y))

            right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            cv2.circle(image, (int(right_elbow.x * image_width), int(right_elbow.y * image_height)), 5, (0, 0, 255), -1)
            right_elbow_coordinates.append((right_elbow.x, right_elbow.y))

        out.write(image)

        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
out.release()  # 출력 파일 닫기
cv2.destroyAllWindows()

비디오 파일을 처리하는데 실패했습니다.


: 

In [15]:
# 팔꿈치 좌표 출력
for i, coord in enumerate(elbow_coordinates):
    print(f"Frame {i+1}: x = {coord[0] * image_width}, y = {coord[1] * image_height}")

Frame 1: x = 630.2509307861328, y = 260.9871554374695
Frame 2: x = 630.8191680908203, y = 259.1338562965393
Frame 3: x = 631.1849975585938, y = 258.43839168548584
Frame 4: x = 631.2295532226562, y = 258.2710003852844
Frame 5: x = 631.2297821044922, y = 257.72945165634155
Frame 6: x = 631.2226104736328, y = 257.31366634368896
Frame 7: x = 631.2545776367188, y = 257.2676610946655
Frame 8: x = 631.4622116088867, y = 257.1494722366333
Frame 9: x = 631.6403961181641, y = 257.171573638916
Frame 10: x = 631.6284942626953, y = 259.61641788482666
Frame 11: x = 631.6498947143555, y = 261.9088053703308
Frame 12: x = 631.5573883056641, y = 263.40041399002075
Frame 13: x = 631.2763595581055, y = 266.4955973625183
Frame 14: x = 627.0973587036133, y = 266.8887662887573
Frame 15: x = 624.9882125854492, y = 266.55529260635376
Frame 16: x = 624.9143600463867, y = 268.8102149963379
Frame 17: x = 625.2378463745117, y = 269.17742013931274
Frame 18: x = 625.4262161254883, y = 271.3859295845032
Frame 19: x =

In [16]:
# 좌표를 데이터프레임에 저장
import pandas as pd
df = pd.DataFrame(elbow_coordinates, columns=["x", "y"])
df.head()

,x,y
0,0.492384,0.362482
1,0.492827,0.359908
2,0.493113,0.358942
3,0.493148,0.358710
4,0.493148,0.357958


In [19]:
output_dir = "D:/Pitch/coordinates"
output_filename = os.path.basename(os.path.splitext(input_file)[0]) + "_elbow_coordinates.csv"
output_file_path = os.path.join(output_dir, output_filename)

df.to_csv(output_file_path, index=False)

In [8]:
import cv2

cap = cv2.VideoCapture(output_file)

if (cap.isOpened()== False): 
    print("비디오를 열 수 없습니다.")
  
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cv2.imshow('Frame', frame)
        # q 키를 눌러 비디오 재생을 중지합니다.
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# q 누르면 비디오 중지됩니다!!! 제발!!!